In [255]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Data

In [256]:
data = pd.read_csv('./data/kaggle_movie/movies_metadata.csv')
data.head()

C:\Users\PIAI\AppData\Local\Temp\ipykernel_14012\3037305875.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('./data/kaggle_movie/movies_metadata.csv')


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [257]:
data.shape

(45466, 24)

In [258]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

In [259]:
# data와 keyword를 병합하기전에 id가 obj형이므로 int형으로 변환하여햐 함. 
# 그전에 아래와 같은 형태의 데이터는 형변환이 안되며 잘못 들어간 데이터이므로 삭제처리함.(내 판단)
# - 문자열을 가진 행 탐색
data[data['id'].str.contains('-')]

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
19730,- Written by Ørnås,0.065736,/ff9qCepilowshEtG2GYWwzt2bs4.jpg,"[{'name': 'Carousel Productions', 'id': 11176}...","[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso...",1997-08-20,0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29503,Rune Balot goes to a casino connected to the ...,1.931659,/zV8bHuSL6WXoD6FWogP9j4x80bL.jpg,"[{'name': 'Aniplex', 'id': 2883}, {'name': 'Go...","[{'iso_3166_1': 'US', 'name': 'United States o...",2012-09-29,0,68.0,"[{'iso_639_1': 'ja', 'name': '日本語'}]",Released,...,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35587,Avalanche Sharks tells the story of a bikini ...,2.185485,/zaSf5OG7V8X8gqFvly88zDdRm46.jpg,"[{'name': 'Odyssey Media', 'id': 17161}, {'nam...","[{'iso_3166_1': 'CA', 'name': 'Canada'}]",2014-01-01,0,82.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,...,22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [260]:
data.drop([35587,19730,29503], axis=0, inplace=True)

In [261]:
data['id'] = data['id'].astype('int')

In [262]:
keyW = pd.read_csv('./data/kaggle_movie/keywords.csv')
keyW

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."
...,...,...
46414,439050,"[{'id': 10703, 'name': 'tragic love'}]"
46415,111109,"[{'id': 2679, 'name': 'artist'}, {'id': 14531,..."
46416,67758,[]
46417,227506,[]


In [263]:
keyW.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46419 entries, 0 to 46418
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        46419 non-null  int64 
 1   keywords  46419 non-null  object
dtypes: int64(1), object(1)
memory usage: 725.4+ KB


In [264]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45463 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45463 non-null  object 
 1   belongs_to_collection  4491 non-null   object 
 2   budget                 45463 non-null  object 
 3   genres                 45463 non-null  object 
 4   homepage               7779 non-null   object 
 5   id                     45463 non-null  int32  
 6   imdb_id                45446 non-null  object 
 7   original_language      45452 non-null  object 
 8   original_title         45463 non-null  object 
 9   overview               44509 non-null  object 
 10  popularity             45460 non-null  object 
 11  poster_path            45077 non-null  object 
 12  production_companies   45460 non-null  object 
 13  production_countries   45460 non-null  object 
 14  release_date           45376 non-null  object 
 15  re

In [265]:
# keyword data와 movie data 연결시키기 (by id)
data = pd.merge(data, keyW, on='id')

## Columns
- genres : 영화 장르
- keywords : 영화의 키워드
- original_language : 영화 언어
- title : 제목
- vote_average : 평점 평균
- vote_count : 평점 카운트
- popularity : 인기도
- overview : 개요 설명


In [266]:
data.shape

(46482, 25)

In [267]:
data.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count', 'keywords'],
      dtype='object')

## Preprocessing

In [268]:
# 사용할 컬럼들 먼저 뽑아주기
data = data[['id','genres','vote_average','vote_count','title','keywords','overview','original_language']]

In [269]:
data.shape

(46482, 8)

### Vote Everage 값 변경
- 많은 사람이 평가할 수록 평점이 5점(만점)이 될 확률은 낮음
- 따라서 평점이 많은 영화에 대해서는 불공평해짐. 이를 방지하기 위한 전처리가 필요

> weighted rating(WR) = (v / (v + m) * R  + (m / (v + m)) * C)

- R = 개별 영화 평점
- V = 개별 영화에 평점을 투표한 횟수
- m = Top n위 안에 들어야 하는 최소 투표
- c = 전체 영화에 대한 평균 평점


In [270]:
# 여기서 m은 460으로 가정하여 진행.
# 전체 데이터가 약 4만 6천개이므로 0.01%정도가 이에 속함.
tmp_m = data['vote_count'].quantile(0.99)
tmp_m

2154.070000000029

In [271]:
tmp_data = data.copy().loc[data['vote_count'] >= tmp_m]
tmp_data.shape


(465, 8)

In [272]:
data.shape

(46482, 8)

In [273]:
# 상위 1% 가정하여 진행
del tmp_data

m = data['vote_count'].quantile(0.99)
# data = data.loc[data['vote_count'] >= m] -> 이 컬럼은 왜 있는지 모르겠다

In [274]:
c = data['vote_average'].mean()

In [275]:
print(f"전체 영화에 대한 평균 평점 : {c}")
print(f"Top 460에 들어야 하는 최소 투표 수 {m}")

전체 영화에 대한 평균 평점 : 5.611883041438665
Top 460에 들어야 하는 최소 투표 수 2154.070000000029


In [276]:
# 앞서 언급했던 불공정한 평점을 보완하기 위한 수식
def weighted_rating(x, m=m, c=c):
    v = x['vote_count']
    R = x['vote_average']
    
    return (v / (v + m) * R) + (m / (m + v) * c)

In [277]:
# 위의 수식을 적용한 값인 score 컬럼 생성 
data['score'] = data.apply(weighted_rating, axis = 1)

# 각 행이 x로 들어가고 각 행에 대한 컬럼 값을 v와 R로 받아와 값을 계산함.


In [278]:
data.head()

,id,genres,vote_average,vote_count,title,keywords,overview,original_language,score
0,862,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",7.7,5415.0,Toy Story,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,...","Led by Woody, Andy's toys live happily in his ...",en,7.105746
1,8844,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",6.9,2413.0,Jumanji,"[{'id': 10090, 'name': 'board game'}, {'id': 1...",When siblings Judy and Peter discover an encha...,en,6.292456
2,15602,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",6.5,92.0,Grumpier Old Men,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392...",A family wedding reignites the ancient feud be...,en,5.648261
3,31357,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",6.1,34.0,Waiting to Exhale,"[{'id': 818, 'name': 'based on novel'}, {'id':...","Cheated on, mistreated and stepped on, the wom...",en,5.619468
4,11862,"[{'id': 35, 'name': 'Comedy'}]",5.7,173.0,Father of the Bride Part II,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n...",Just when George Banks has recovered from his ...,en,5.618434


In [279]:
data.shape

(46482, 9)

### Genres, Keywords 변환

In [280]:
data[['genres','keywords']].head()

# dic 구조이지만 전체가 문자열로 들어가있기에 이를 해결해야 함. 

,genres,keywords
0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...","[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...","[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...","[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,"[{'id': 35, 'name': 'Comedy'}]","[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


In [281]:
# ast 내의 literal_eval 패키지 이용 -> list와 dic 형태로 변환됨
data['genres'] = data['genres'].apply(literal_eval)
data['keywords'] = data['keywords'].apply(literal_eval)

In [282]:
data[['genres','keywords']].head()
# list, dic 형태로 변환됨

,genres,keywords
0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...","[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...","[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...","[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,"[{'id': 35, 'name': 'Comedy'}]","[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


In [283]:
# id를 제거한 후 name만 뽑아내면 됨 (장르가 무엇인지, 키워드가 무엇인지가 중요하므로 )
data['genres'] = data['genres'].apply (lambda x: [d['name'] for d in x]).apply(lambda x : " ".join(x))
data['keywords'] = data['keywords'].apply(lambda x : [d['name'] for d in x]).apply(lambda x : " ".join(x))
# x는 data['genres']이고 d는 리스트 내 하나의 딕셔너리 데이터를 의미하는 듯함(내생각)

In [284]:
data.head(2)

,id,genres,vote_average,vote_count,title,keywords,overview,original_language,score
0,862,Animation Comedy Family,7.7,5415.0,Toy Story,jealousy toy boy friendship friends rivalry bo...,"Led by Woody, Andy's toys live happily in his ...",en,7.105746
1,8844,Adventure Fantasy Family,6.9,2413.0,Jumanji,board game disappearance based on children's b...,When siblings Judy and Peter discover an encha...,en,6.292456


In [285]:
# 데이터 저장
data.to_csv('./data/kaggle_movie/main_data.csv', index=False)

# Content Based Filtering
- 비슷한 콘텐츠를 사용자에게 추천하는 것.
- '장르'를 이용해서 추천시스템을 진행 

## Vecotrization

In [286]:
movie_data = pd.read_csv('./data/kaggle_movie/main_data.csv')
movie_data = movie_data.loc[movie_data['original_language'] == 'en', :]
movie_data = movie_data[['id','title','original_language','genres']]

print(movie_data.shape)
movie_data.head()

(32852, 4)


,id,title,original_language,genres
0,862,Toy Story,en,Animation Comedy Family
1,8844,Jumanji,en,Adventure Fantasy Family
2,15602,Grumpier Old Men,en,Romance Comedy
3,31357,Waiting to Exhale,en,Comedy Drama Romance
4,11862,Father of the Bride Part II,en,Comedy


In [287]:
# 영화 id에 따른 keyword값
movie_keyword = pd.read_csv('./data/kaggle_movie/keywords.csv')
print(movie_keyword.shape)
movie_keyword.head()

(46419, 2)


,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


In [288]:
movie_data.id = movie_data.id.astype(int)
movie_keyword.id = movie_keyword.id.astype(int)

# id 기준으로 df 합쳐주기 
movie_data = pd.merge(movie_data, movie_keyword, on='id')
print(movie_data.shape)
movie_data.head()

(34022, 5)


,id,title,original_language,genres,keywords
0,862,Toy Story,en,Animation Comedy Family,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,Jumanji,en,Adventure Fantasy Family,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,Grumpier Old Men,en,Romance Comedy,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,Waiting to Exhale,en,Comedy Drama Romance,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,Father of the Bride Part II,en,Comedy,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


In [289]:
# 앞선 전처리 방식과 동일한 방식으로 전처리 진행
movie_data['keywords'] = movie_data['keywords'].apply(literal_eval)
movie_data['keywords'] = movie_data['keywords'].apply(lambda x : [d['name'] for d in x]).apply(lambda x : " ".join(x))

### TF - IDF 벡터화
- Term Frequency - Inverse Document Frequency
    - 단어의 빈도와 역 문서빈도를 사용하여 DTM내의 각 단어들마다 중요한 정도를 가중치로 주는 방법
- 주로 문서의 유사도를 구하는 작업, 검색 시스템에서 검색 결과의 중요도를 정하는 작업, 문서 내에서 특정 단어의 중요도를 구하는 작업 등에 쓰일 수 있다. 
- TF와 IDF를 곱한 값을 의미하는 데, 문서를 d, 단어를 t, 문서의 총 개수를 n이라고 표현할 때 다음과 같이 정의 가능하다.
    - **TF(d,t)** : 특정 문서 d에서의 특정 단어 t의 등장 횟수 
    - **DF(t)** : 특정 단어 t가 등장한 문서의 수
    - **IDF(d,t)** : df(t)에 반비례하는 수

In [290]:
movie_data.isnull().sum()

id                      0
title                   5
original_language       0
genres               1847
keywords                0
dtype: int64

In [291]:
# null 데이터가 있으면 벡터화가 안되므로 전부 삭제 처리
movie_data.dropna(axis=0, inplace = True)

In [292]:
movie_data.isnull().sum()

id                   0
title                0
original_language    0
genres               0
keywords             0
dtype: int64

In [293]:
tfidf_vector = TfidfVectorizer()
# 장르와 키워드를 하나로 합친 후 tf-idf 벡터로 만들어 줌.
tfidf_matrix = tfidf_vector.fit_transform(movie_data['genres'] + " " + movie_data['keywords']).toarray()
tfidf_matrix_feature = tfidf_vector.get_feature_names()

c:\Users\PIAI\Anaconda3\envs\yujin\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [294]:
tfidf_matrix.shape
# shape = (train 문장 수 , 사전의 단어 수)
# 장르와 키워드를 하나의 문장으로 합쳤기때문에 이는 하나의 row가 하나의 문장으로 들어간 것과 같다.
# 따라서 32170개의 행(=문장)과 11429개의 단어로 이루어진 matrix가 생성됨
# 이는 32,170개의 영화를 표현하기 위해서 총 11,429개의 단어가 사용되었음을 의미

(32170, 11429)

In [295]:
tfidf_df = pd.DataFrame(tfidf_matrix, columns = tfidf_matrix_feature, index = movie_data.title)
print(tfidf_df.shape)
tfidf_df.head()
# 각 row는 한 영화에 대한 정보를 나타내므로 영화 타이틀을 인덱스로 가져옴. 
 

(32170, 11429)


,077,10,11,13,1500s,15th,16th,17th,1812,18th,...,βάφτηκε,γη,κόκκινο,το,χώμα,миньоны,卧底肥妈,绝地奶霸,自然界大事件,超级妈妈
title,,,,,,,,,,,,,,,,,,,,,
Toy Story,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Jumanji,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Grumpier Old Men,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Waiting to Exhale,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Father of the Bride Part II,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 유사도 구하기

In [296]:
%%time
# 코사인 유사도 사용 - 영화 개수(n)만큼 n x n  matrix 형태의 output
cosine_sim = cosine_similarity(tfidf_df)

CPU times: total: 12min 43s
Wall time: 3min 33s


In [297]:
cosine_sim.shape

(32170, 32170)

In [298]:
cosine_sim_df = pd.DataFrame(cosine_sim, index = movie_data.title, columns = movie_data.title)
print(cosine_sim_df)
cosine_sim_df.head()

title                        Toy Story   Jumanji  Grumpier Old Men  \
title                                                                
Toy Story                     1.000000  0.041258          0.008445   
Jumanji                       0.041258  1.000000          0.000000   
Grumpier Old Men              0.008445  0.000000          1.000000   
Waiting to Exhale             0.006716  0.064435          0.034941   
Father of the Bride Part II   0.005418  0.000000          0.010592   
...                                ...       ...               ...   
Shadow of the Blair Witch     0.000000  0.000000          0.000000   
The Burkittsville 7           0.000000  0.000000          0.000000   
Caged Heat 3000               0.000000  0.000000          0.000000   
Robin Hood                    0.000000  0.000000          0.105827   
Betrayal                      0.000000  0.000000          0.000000   

title                        Waiting to Exhale  Father of the Bride Part II  \
title     

title,Toy Story,Jumanji,Grumpier Old Men,Waiting to Exhale,Father of the Bride Part II,Heat,Sabrina,Tom and Huck,Sudden Death,GoldenEye,...,The Devilish Tenant,Pooh's Heffalump Halloween Movie,Deep Hearts,The Morning After,House of Horrors,Shadow of the Blair Witch,The Burkittsville 7,Caged Heat 3000,Robin Hood,Betrayal
title,,,,,,,,,,,,,,,,,,,,,
Toy Story,1.000000,0.041258,0.008445,0.006716,0.005418,0.000000,0.006245,0.058795,0.000000,0.000000,...,0.011796,0.050876,0.0,0.050444,0.028207,0.0,0.0,0.0,0.000000,0.000000
Jumanji,0.041258,1.000000,0.000000,0.064435,0.000000,0.000000,0.000000,0.164725,0.027794,0.011241,...,0.052657,0.028225,0.0,0.000000,0.039243,0.0,0.0,0.0,0.000000,0.000000
Grumpier Old Men,0.008445,0.000000,1.000000,0.034941,0.010592,0.000000,0.032489,0.000000,0.000000,0.000000,...,0.023061,0.000000,0.0,0.098613,0.000000,0.0,0.0,0.0,0.105827,0.000000
Waiting to Exhale,0.006716,0.064435,0.034941,1.000000,0.093272,0.003626,0.062706,0.026460,0.000000,0.000000,...,0.018340,0.000000,0.0,0.133145,0.000000,0.0,0.0,0.0,0.119512,0.036339
Father of the Bride Part II,0.005418,0.000000,0.010592,0.093272,1.000000,0.000000,0.037576,0.000000,0.000000,0.000000,...,0.014795,0.000000,0.0,0.063268,0.000000,0.0,0.0,0.0,0.000000,0.000000


## Comtent Based Recommend
- target title : 추천 결과를 조회할 영화 제목에 따라서 코사인 유사도를 구한 matrix에서 유사도 데이터를 가져옴
- 유사도 데이터 중 가장 유사도 값이 큰 데이터를 가져옴
    - 가져올 때 top k개를 가져옴
- 해당 추천 값 출력

In [304]:
cosine_sim_df.loc[:,'The Dark Knight Rises'].values.reshape(1,-1).argsort()[:,::-1].flatten()[1:10+1]

array([10160, 20497,  8362,  1278,   533, 20346,  4442, 30240, 29285,
        1135], dtype=int64)

In [307]:
def genre_recommendations(target_title, matrix, items, k=10):
    recom_idx = matrix.loc[:, target_title].values.reshape(1,-1).argsort()[:, ::-1].flatten()[1:k+1]
    recom_title = items.iloc[recom_idx, :].title.values
    recom_genre = items.iloc[recom_idx, :].genres.values
    target_title_list = np.full(len(range(k)), target_title)
    target_genre_list = np.full(len(range(k)), items[items.title == target_title].genres.values)
    
    d = {
        'target_title' : target_title_list,
        'target_genre' : target_genre_list,
        'recom_title' : recom_title,
        'recom_genre' : recom_genre
    }
    return pd.DataFrame(d)

In [308]:
genre_recommendations('The Dark Knight Rises', cosine_sim_df, movie_data)

,target_title,target_genre,recom_title,recom_genre
0,The Dark Knight Rises,Action Crime Drama Thriller,The Dark Knight,Drama Action Crime Thriller
1,The Dark Knight Rises,Action Crime Drama Thriller,The Burglar,Crime Drama
2,The Dark Knight Rises,Action Crime Drama Thriller,Batman Begins,Action Crime Drama
3,The Dark Knight Rises,Action Crime Drama Thriller,Batman & Robin,Action Crime Fantasy
4,The Dark Knight Rises,Action Crime Drama Thriller,Batman,Fantasy Action
5,The Dark Knight Rises,Action Crime Drama Thriller,Raffles,Adventure Comedy Crime Drama History Romance T...
6,The Dark Knight Rises,Action Crime Drama Thriller,Hero at Large,Action Comedy Drama
7,The Dark Knight Rises,Action Crime Drama Thriller,DC Showcase: Catwoman,Action Adventure Animation Science Fiction
8,The Dark Knight Rises,Action Crime Drama Thriller,DC Super Hero Girls: Hero of the Year,Animation
9,The Dark Knight Rises,Action Crime Drama Thriller,Batman Returns,Action Fantasy
